In [1]:
import numpy as np
import scipy.io as io
from nonnegfac.nmf import NMF

In [44]:
patent_data = io.loadmat('./wbd_1061v0(behavior)IE')

In [45]:
patent_data

{'V': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Oct 09 12:48:18 2017',
 '__version__': '1.0',
 'patentnum': array([[array([[5712454]])],
        [array([[5497941]])],
        [array([[5517952]])],
        ...,
        [array([[5119853]])],
        [array([[9637261]])],
        [array([[1111111]])]], dtype=object),
 'titlecell': array([[array([u'Exhaust system in internal combustion engine'], dtype='<U242')],
        [array([u'System for controlling the temperature of the air in a cabin for an     engine-electric motor hybrid car'],
       dtype='<U242')],
        [array([u'Rotating shuttle engines with integral valving'], dtype='<U242')],
        ...,
        [array([u'Apparatus for filling ca

In [47]:
A = patent_data['V']
patent_no = patent_data['patentnum']
patent_titles = patent_data['titlecell']
words_ref = patent_data['words'][0]

In [138]:
patent_no[0][0][0][0]

5712454

In [10]:
k = 3
W, H, info = NMF().run(A, k)

[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 1061, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random", 
    "k": 3, 
    "max_iter": 50, 
    "max_time": Infinity, 
    "verbose": 0
}
[NMF] Completed: 
{
    "elapsed": 0.17799997329711914, 
    "iterations": 50, 
    "norm_A": 199.33181515533371, 
    "rel_error": 0.8626993596895095
}


In [11]:
print A.shape
print W.shape
print H.shape
print info

(262L, 1061L)
(262L, 3L)
(1061L, 3L)
{'info': {'A_dim_1': 262L, 'A_dim_2': 1061L, 'init': 'uniform_random', 'verbose': 0, 'max_time': inf, 'alg': "<class 'nonnegfac.nmf.NMF'>", 'k': 3, 'max_iter': 50, 'A_type': "<type 'numpy.ndarray'>"}, 'final': {'rel_error': 0.8626993596895095, 'norm_A': 199.33181515533371, 'iterations': 50, 'elapsed': 0.17799997329711914}}


In [12]:
print W.shape
print H.shape

(262L, 3L)
(1061L, 3L)


In [13]:
# return top keywords per new topic
top_word_n = 5
topic_words = np.zeros((k, top_word_n))
for i in range(k):
    topic = W[:, i]
    topword_idxs = np.argsort(topic)[::-1][:top_word_n]
    # debug 
    # print np.sort(topic)[::-1][:top_word_n]
    # print [words[idx] for idx in topword_idxs]
    topic_words[i] = topword_idxs

In [14]:
patent_topic = np.argmax(H, axis=1)
print patent_topic[:10]
print H[:10]

[1 2 0 1 0 1 2 2 0 0]
[[0.80519196 2.89584169 0.        ]
 [2.93930076 0.         3.52639691]
 [1.60128586 0.81202181 0.26705332]
 [1.11819788 2.13813218 0.        ]
 [2.09290012 1.76805607 0.94375494]
 [0.         3.53673103 0.07908796]
 [0.         0.08763235 2.38580421]
 [0.         0.85880369 1.00629237]
 [3.3677761  2.30071285 0.        ]
 [0.99135142 0.89568026 0.        ]]


In [18]:
for i in range(k):
    topic_idx = i
    patent_idx = np.where(patent_topic==i)
    patent_nums = [patent_no[j] for j in patent_idx]
    patent_data = A[:, patent_idx]
    print patent_idx[0].shape
    print patent_data.shape

(247L,)
(262L, 1L, 247L)
(529L,)
(262L, 1L, 529L)
(285L,)
(262L, 1L, 285L)


In [ ]:
# takes a patent matrix of N by vocab_n matrix, return all patents if terminal condition is met
# else return left and right components, patents that belong to them, and 
# their respective vocabs words, 



In [65]:
topic_idx = 0
patent_idx = np.where(patent_topic==i)
patent_nums = [patent_no[j] for j in patent_idx]
patent_data = A[:, patent_idx]
print patent_idx[0].shape
print patent_data.shape



(285L,)
(262L, 1L, 285L)


In [66]:
w, h, d = patent_data.shape

In [67]:
patent_data.reshape(w, d).shape

(262L, 285L)

In [30]:
patent_nums[0][:10]

array([[array([[5497941]])],
       [array([[8869772]])],
       [array([[8387582]])],
       [array([[8242655]])],
       [array([[7683499]])],
       [array([[7231894]])],
       [array([[6948466]])],
       [array([[7827967]])],
       [array([[6718928]])],
       [array([[7721551]])]], dtype=object)

In [35]:
patent_no[0]

array([array([[5712454]])], dtype=object)

In [37]:
patent_no[patent_idx].shape

(285L, 1L)

In [181]:
def terminal_fn(data):
    n = 50
    r, c = data.shape
    return c < n; 

In [182]:
def nmf(data, k, term_fn, patent_no_ref, patent_names_ref, word_ref):
    
    print 'running nmf with data of shape', data.shape, ' and ', k, ' clusters'
    result = {}
    
    # if we reached terminal condition, return our leaf patents
    if term_fn(data):
        result['patents'] = patent_no_ref[0]
        result['no patents'] = data.shape[1]
        return result
    
    # run nmf on data
    W, H, info = NMF().run(data, k)
 
    # get topic words per topic
    top_word_n = 5
    topic_words = []
    topic_word_idxs = []
    for i in range(k):
        topic = W[:, i]
        topword_idxs = np.argsort(topic)[::-1][:top_word_n]
        # debug 
        # print np.sort(topic)[::-1][:top_word_n]
        # print [words[idx] for idx in topword_idxs]
        topic_word_idxs.append(topword_idxs)
        topic_words.append([word_ref[x] for x in topword_idxs])
        
    result['topic_keyword_idxs'] = topic_word_idxs
    result['topic_keywords'] = topic_words

    # get topic index of each patent and their respective document number and data
    patent_topics = np.argmax(H, axis=1)
    print 'patent topics', patent_topics[:10]
    print 'patent topics shape', patent_topics.shape
    
    child_patent_data = []
    child_patent_no_ref = []
    
#     for i in range(k):
#         print 'child data is of shape', child
        
    for i in range(k):
        topic_idx = i
        patent_idx = np.where(patent_topics==i)
        child_patent_no_ref.append([patent_no[j] for j in patent_idx])
        child_data = data[:, patent_idx]
        w, h, d = child_data.shape
        child_data = child_data.reshape(w, d)
        child_patent_data.append(child_data) 
    
    child_data = []
    for i in range(k):
        print child_patent_data[i].shape
        child_data.append(nmf(child_patent_data[i], k, term_fn, child_patent_no_ref[i], patent_names_ref, word_ref))
        
    result['child_data'] = child_data
    
    return result     

In [183]:
import time
begin = time.time()
result = nmf(A, 2, terminal_fn, patent_no, patent_titles, words_ref)
print 'took ', time.time() - begin, 'seconds'

running nmf with data of shape (262L, 1061L)  and  2  clusters
[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 1061, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random", 
    "k": 2, 
    "max_iter": 50, 
    "max_time": Infinity, 
    "verbose": 0
}
[NMF] Completed: 
{
    "elapsed": 0.09900021553039551, 
    "iterations": 50, 
    "norm_A": 199.33181515533371, 
    "rel_error": 0.8829587744207108
}
patent topics [1 0 0 1 0 1 0 1 1 1]
patent topics shape (1061L,)
(262L, 492L)
running nmf with data of shape (262L, 492L)  and  2  clusters
[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 492, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random", 
    "k": 2, 
    "max_iter": 50, 
    "max_time": Infinity, 
    "verbose": 0
}
[NMF] Completed: 
{
    "elapsed": 0.10500001907348633, 
    "iterations": 50, 
    "norm_A": 148.8675565252701, 
    "rel_error": 0.

[NMF] Completed: 
{
    "elapsed": 0.06699991226196289, 
    "iterations": 50, 
    "norm_A": 35.51926598782856, 
    "rel_error": 0.74652083813361
}
patent topics [1 1 1 0 1 1 1 1 1 1]
patent topics shape (51L,)
(262L, 4L)
running nmf with data of shape (262L, 4L)  and  2  clusters
(262L, 47L)
running nmf with data of shape (262L, 47L)  and  2  clusters
(262L, 20L)
running nmf with data of shape (262L, 20L)  and  2  clusters
(262L, 24L)
running nmf with data of shape (262L, 24L)  and  2  clusters
(262L, 20L)
running nmf with data of shape (262L, 20L)  and  2  clusters
(262L, 61L)
running nmf with data of shape (262L, 61L)  and  2  clusters
[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 61, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random", 
    "k": 2, 
    "max_iter": 50, 
    "max_time": Infinity, 
    "verbose": 0
}
[NMF] Completed: 
{
    "elapsed": 0.0970001220703125, 
    "iterations": 50, 
    "norm_A": 48.5

[NMF] Completed: 
{
    "elapsed": 0.08000016212463379, 
    "iterations": 50, 
    "norm_A": 75.41432514022216, 
    "rel_error": 0.7980160375768494
}
patent topics [1 0 1 0 1 1 0 1 1 0]
patent topics shape (176L,)
(262L, 90L)
running nmf with data of shape (262L, 90L)  and  2  clusters
[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 90, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random", 
    "k": 2, 
    "max_iter": 50, 
    "max_time": Infinity, 
    "verbose": 0
}
[NMF] Completed: 
{
    "elapsed": 0.0690000057220459, 
    "iterations": 50, 
    "norm_A": 52.30457525389743, 
    "rel_error": 0.7809997163410708
}
patent topics [0 0 1 0 0 0 0 0 0 1]
patent topics shape (90L,)
(262L, 63L)
running nmf with data of shape (262L, 63L)  and  2  clusters
[NMF] Running: 
{
    "A_dim_1": 262, 
    "A_dim_2": 63, 
    "A_type": "<type 'numpy.ndarray'>", 
    "alg": "<class 'nonnegfac.nmf.NMF'>", 
    "init": "uniform_random

In [186]:
result['child_data'][1]['child_data'][0]

{'child_data': [{'child_data': [{'child_data': [{'child_data': [{'child_data': [{'no patents': 4L,
           'patents': array([[array([[6434925]])],
                  [array([[3974811]])],
                  [array([[6557514]])],
                  [array([[7652828]])]], dtype=object)},
          {'no patents': 46L, 'patents': array([[array([[5712454]])],
                  [array([[5497941]])],
                  [array([[5517952]])],
                  [array([[4854277]])],
                  [array([[9556782]])],
                  [array([[9482122]])],
                  [array([[8869772]])],
                  [array([[8387582]])],
                  [array([[4979473]])],
                  [array([[6345958]])],
                  [array([[5154593]])],
                  [array([[3937589]])],
                  [array([[5997245]])],
                  [array([[8242655]])],
                  [array([[7683499]])],
                  [array([[7231894]])],
                  [array([[6691657]])],
   